In [1]:
import scanpy as sc
import squidpy as sq

import numpy as np
import pandas as pd

In [2]:
adata = sc.read_h5ad('../datasets/multiome/multiome_rna.h5ad')
sc.pp.normalize_total(adata)
sc.pp.log1p(adata)
adata

AnnData object with n_obs × n_vars = 2535 × 2000
    obs: 'cell_type'
    var: 'gene_ids', 'feature_types'
    uns: 'log1p'
    obsm: 'spatial'

In [3]:
adata.X = adata.X.toarray()

In [4]:
from model import SpiceMix
from sklearn.neighbors import kneighbors_graph
import torch
K, num_pcs, n_neighbors, res_lo, res_hi = 20, 20, 10, .5, 2.
context = dict(device='cuda:6', dtype=torch.float64)
repli_list = ['multiome']

A = kneighbors_graph(adata.obsm['spatial'],n_neighbors = 10)
from pytorch_lightning.utilities.seed import seed_everything
for i in range(10):
    seed_everything(i)
    print(i)
    obj = SpiceMix(
    K=20,
    lambda_Sigma_x_inv=1e-6, power_Sigma_x_inv=2,
    repli_list=repli_list,
    context=context,
    context_Y=context,
    )
    obj.Ys = [adata.X]
    obj.Gs = [2000]
    obj.GG = 2000
    obj.Ys = [G / obj.GG * K * Y / Y.sum(1).mean() for Y, G in zip(obj.Ys, obj.Gs)]
    obj.Ys = [torch.tensor(Y, **context).pin_memory() 
               if context['device'] != 'cpu' and context['device'] == 'cpu' else
               torch.tensor(Y, **context) for Y in obj.Ys]
    obj.Es = [np.vstack((A.tocoo().row, A.tocoo().col)).T]
    obj.Es_isempty = [True, True, True]

    obj.initialize(method='kmeans',random_state=i)
    for iiter in range(10):
        obj.estimate_weights(iiter=iiter, use_spatial=[False]*obj.num_repli)
        obj.estimate_parameters(iiter=iiter, use_spatial=[False]*obj.num_repli)
    obj.initialize_Sigma_x_inv()
    for iiter in range(1, 101):
        obj.estimate_parameters(iiter=iiter, use_spatial=[True]*obj.num_repli)
        obj.estimate_weights(iiter=iiter, use_spatial=[True]*obj.num_repli)
    adata.obsm['spicemix'+str(i)] = torch.cat(obj.Xs).cpu().numpy()
    np.save('../Benchmarking_results/multiome/'+'spicemix'+str(i)+'.npy',adata.obsm['spicemix'+str(i)])

Global seed set to 0


0


/usr/local/lib/python3.8/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
2024-04-02 04:05:23:INFO:Updating latent states
2024-04-02 04:05:24:INFO:Updating model parameters
2024-04-02 04:05:24:INFO:Q value = nan
2024-04-02 04:05:24:INFO:Updating latent states
2024-04-02 04:05:25:INFO:Updating model parameters
2024-04-02 04:05:25:INFO:Q value = nan
2024-04-02 04:05:25:INFO:Updating latent states
2024-04-02 04:05:25:INFO:Updating model parameters
2024-04-02 04:05:25:INFO:Q value = nan
2024-04-02 04:05:25:INFO:Updating latent states
2024-04-02 04:05:25:INFO:Updating model parameters
2024-04-02 04:05:26:INFO:Q value = nan
2024-04-02 04:05:26:INFO:Updating latent states
2024-04-02 04:05:26:INFO:Updating model parameters
2024-04-02 04:05:26:INFO:Q value = nan
2024-04-02 04:05:26:INFO:Updating latent states
2024-04-02 04:05:26:INFO:Updat

2024-04-02 04:10:09:INFO:Q value = -8481801.653958587
2024-04-02 04:10:10:INFO:Updating latent states
2024-04-02 04:10:10:INFO:Updating model parameters
2024-04-02 04:10:22:INFO:Q value = -8481706.978429403
2024-04-02 04:10:22:INFO:Updating latent states
2024-04-02 04:10:22:INFO:Updating model parameters
2024-04-02 04:10:27:INFO:Q value = -8481876.862426385
2024-04-02 04:10:27:INFO:Updating latent states
2024-04-02 04:10:27:INFO:Updating model parameters
2024-04-02 04:10:31:INFO:Q value = -8481887.648747437
2024-04-02 04:10:31:INFO:Updating latent states
2024-04-02 04:10:31:INFO:Updating model parameters
2024-04-02 04:10:36:INFO:Q value = -8481891.236711869
2024-04-02 04:10:36:INFO:Updating latent states
2024-04-02 04:10:36:INFO:Updating model parameters
2024-04-02 04:10:41:INFO:Q value = -8481875.774026262
2024-04-02 04:10:41:INFO:Updating latent states
2024-04-02 04:10:41:INFO:Updating model parameters
2024-04-02 04:10:47:INFO:Q value = -8481825.316410864
2024-04-02 04:10:47:INFO:Upd

2024-04-02 04:15:11:INFO:Updating model parameters
2024-04-02 04:15:15:INFO:Q value = -8480467.478238272
2024-04-02 04:15:15:INFO:Updating latent states
2024-04-02 04:15:15:INFO:Updating model parameters
2024-04-02 04:15:17:INFO:Q value = -8480420.955323279
2024-04-02 04:15:17:INFO:Updating latent states
2024-04-02 04:15:17:INFO:Updating model parameters
2024-04-02 04:15:22:INFO:Q value = -8480369.186227152
2024-04-02 04:15:22:INFO:Updating latent states
Global seed set to 1
/usr/local/lib/python3.8/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


...

2024-04-02 05:48:16:INFO:Q value = -8474738.598806461
2024-04-02 05:48:16:INFO:Updating latent states
2024-04-02 05:48:16:INFO:Updating model parameters
2024-04-02 05:48:21:INFO:Q value = -8474729.139413008
2024-04-02 05:48:21:INFO:Updating latent states
Global seed set to 9
/usr/local/lib/python3.8/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


9


2024-04-02 05:48:23:INFO:Updating latent states
2024-04-02 05:48:23:INFO:Updating model parameters
2024-04-02 05:48:24:INFO:Q value = nan
2024-04-02 05:48:24:INFO:Updating latent states
2024-04-02 05:48:24:INFO:Updating model parameters
2024-04-02 05:48:24:INFO:Q value = nan
2024-04-02 05:48:24:INFO:Updating latent states
2024-04-02 05:48:24:INFO:Updating model parameters
2024-04-02 05:48:24:INFO:Q value = nan
2024-04-02 05:48:24:INFO:Updating latent states
2024-04-02 05:48:24:INFO:Updating model parameters
2024-04-02 05:48:25:INFO:Q value = nan
2024-04-02 05:48:25:INFO:Updating latent states
2024-04-02 05:48:25:INFO:Updating model parameters
2024-04-02 05:48:25:INFO:Q value = nan
2024-04-02 05:48:25:INFO:Updating latent states
2024-04-02 05:48:25:INFO:Updating model parameters
2024-04-02 05:48:25:INFO:Q value = nan
2024-04-02 05:48:25:INFO:Updating latent states
2024-04-02 05:48:25:INFO:Updating model parameters
2024-04-02 05:48:26:INFO:Q value = nan
2024-04-02 05:48:26:INFO:Updating 

2024-04-02 05:52:46:INFO:Updating latent states
2024-04-02 05:52:46:INFO:Updating model parameters
2024-04-02 05:52:50:INFO:Q value = -8479805.50790238
2024-04-02 05:52:50:INFO:Updating latent states
2024-04-02 05:52:50:INFO:Updating model parameters
2024-04-02 05:52:54:INFO:Q value = -8479723.045257501
2024-04-02 05:52:54:INFO:Updating latent states
2024-04-02 05:52:54:INFO:Updating model parameters
2024-04-02 05:53:03:INFO:Q value = -8479683.025257224
2024-04-02 05:53:03:INFO:Updating latent states
2024-04-02 05:53:03:INFO:Updating model parameters
2024-04-02 05:53:07:INFO:Q value = -8479740.643832449
2024-04-02 05:53:07:INFO:Updating latent states
2024-04-02 05:53:07:INFO:Updating model parameters
2024-04-02 05:53:11:INFO:Q value = -8479699.700872283
2024-04-02 05:53:11:INFO:Updating latent states
2024-04-02 05:53:11:INFO:Updating model parameters
2024-04-02 05:53:13:INFO:Q value = -8479692.595455602
2024-04-02 05:53:13:INFO:Updating latent states
2024-04-02 05:53:13:INFO:Updating m

2024-04-02 05:57:46:INFO:Q value = -8476988.526194742
2024-04-02 05:57:46:INFO:Updating latent states
2024-04-02 05:57:46:INFO:Updating model parameters
2024-04-02 05:57:53:INFO:Q value = -8476957.097582025
2024-04-02 05:57:53:INFO:Updating latent states
